In [1]:
import pandas as pd
import collections
import pickle

In [2]:
shows = pd.read_pickle('pulls/shows_2020-12-17.pickle')
songs = pd.read_pickle('pulls/songs_2020-12-17.pickle')
live_songs = pd.read_pickle('pulls/live_songs_2020-12-17.pickle')

### Export Song Abbreviations

In [3]:
code_to_name = songs['name'].to_dict()
name_to_code = {code_to_name[i] : i for i in code_to_name}
codes = list(collections.OrderedDict(sorted(name_to_code.items())).items())

In [4]:
table = {}
for i in range(100):
    table[i] = []
for i in range(len(codes)):
    table[i % 100].append(codes[i])
for i in range(100):
    table[i] = [item for sublist in table[i] for item in sublist]
table = list(table.values())

In [5]:
pd.DataFrame(table).to_csv('song_codes.csv')